# Exercise: Triangle
The purpose of this exercise is to use flopy and triangle to build a triangular mesh.

In [ ]:
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy
from flopy.utils.triangle import Triangle as Triangle
%matplotlib inline


In [ ]:
import geopandas as gp

## read a shapefile of Switzerland including the cantons into `geopandas`

In [ ]:
ch_cantons = gp.read_file('gadm36_CHE_shp/gadm36_CHE_1.shp')

In [ ]:
ch_cantons.geometry

## project to an equal-area projection appropriate for Switzerland

In [ ]:
ch_cantons.geometry = ch_cantons.geometry.to_crs(21781)

## now let's make an outline to use for the whole domain


In [ ]:
switzerland_outline = ch_cantons.copy()
switzerland_outline.geometry = switzerland_outline.geometry.convex_hull
switzerland_outline = switzerland_outline.dissolve(by='GID_0')

In [ ]:
print(len(switzerland_outline.geometry[0].boundary.xy[0]))
#switzerland_outline.geometry=switzerland_outline.geometry.to_crs(21781)
switzerland_outline.geometry[0]

In [ ]:
switzerland_outline.geometry[0].boundary.xy

## now we need to get lists of points to provide to triangle

In [ ]:
domainpoly = []
for x,y in zip(switzerland_outline.geometry[0].boundary.xy[0],switzerland_outline.geometry[0].boundary.xy[1]):
    domainpoly.append((float(x), float(y)))
maximum_area = 100000000

In [ ]:
len(switzerland_outline.geometry[0].boundary.xy[0])

## make a `triangle` object form which we can build an unstructured grid

In [ ]:
model_ws = './triangle'
if os.path.exists(model_ws):
    shutil.rmtree(model_ws)
os.makedirs(model_ws)

tri = Triangle(maximum_area=maximum_area, angle=33, model_ws=model_ws, 
               exe_name='triangle')
tri.add_polygon(domainpoly)
tri.build(verbose=False)
fig = plt.figure(figsize=(15,15))
ax = plt.subplot(1, 1, 1, aspect='equal')
pc = tri.plot(ax=ax)

In [ ]:
tri.ncpl

### we can visualize which are the boundary cells 

In [ ]:
ibd = tri.get_boundary_marker_array()
ibd = np.ma.masked_equal(ibd, 0)
fig = plt.figure(figsize=(15,15))
ax = plt.subplot(1, 1, 1, aspect='equal')
pc = tri.plot(a=ibd, cmap='jet')
plt.colorbar(pc, shrink=0.5)

### now find the edges so we can set their constant head values equal to a boundary value

In [ ]:
edgenodes = []
for iedge in range(len(domainpoly)):
    nodes = tri.get_edge_cells(iedge)
    for n in nodes:
        if n not in edgenodes:
            edgenodes.append(n)

## now build a modflow model using this grid

In [ ]:
name = 'mf'
sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                             exe_name='mf6',
                             sim_ws=model_ws)
tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                             perioddata=[[1.0, 1, 1.]])
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
ims = flopy.mf6.ModflowIms(sim, print_option='SUMMARY', complexity='complex', 
                           outer_hclose=1.e-5, inner_hclose=1.e-4)
cell2d = tri.get_cell2d()
vertices = tri.get_vertices()
xcyc = tri.get_xcyc()
nlay = 1
ncpl = tri.ncpl
nvert = tri.nvert
top = 1.
hk=0.8
botm = [0.]
dis = flopy.mf6.ModflowGwfdisv(gwf, nlay=nlay, ncpl=ncpl, nvert=nvert,
                               top=top, botm=botm, 
                               vertices=vertices, cell2d=cell2d)
npf = flopy.mf6.ModflowGwfnpf(gwf, k=hk,xt3doptions=[True], 
                              save_specific_discharge=True)
ic = flopy.mf6.ModflowGwfic(gwf)

chdlist = []
for icpl in edgenodes:
    h = 0.
    chdlist.append([(0, icpl), h])
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chdlist)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=0.01)
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord='{}.cbc'.format(name),
                            head_filerecord='{}.hds'.format(name),
                            saverecord=[('HEAD', 'LAST'),
                                        ('BUDGET', 'LAST')],
                            printrecord=[('HEAD', 'LAST'),
                                         ('BUDGET', 'LAST')])
sim.write_simulation()
success, buff = sim.run_simulation(report=True)

## visualize the head solution

In [ ]:
fname = os.path.join(model_ws, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
fname = os.path.join(model_ws, name + '.cbc')
bdobj = flopy.utils.CellBudgetFile(fname, precision='double', verbose=False)
spdis = bdobj.get_data(text='DATA-SPDIS')[0]

fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
h=tri.plot(ax=ax, a=head[0, 0, :], cmap='jet', alpha=.3)
plt.colorbar(h, shrink=.5)

In [ ]:
# with vectors
fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
pmv = flopy.plot.PlotMapView(model=gwf, ax=ax)
pmv.plot_array(head[0, :, :], cmap='jet', edgecolor='0.1', alpha=0.3)
pmv.plot_specific_discharge(spdis)

# More complex geometry options

## first let's make a hole

In [ ]:
# note that we can only work with polygon types - not multipolygons

In [ ]:
ch_cantons[['NAME_1','geometry']]

# pick a canton to exclude from the grid

In [ ]:
hole_poly_df = ch_cantons.loc[ch_cantons.NAME_1=='Schwyz']

## `geopandas` can help us simplify -- we don't want too many points

In [ ]:
refine_tolerance = 150

In [ ]:
print(len(hole_poly_df.geometry.values[0].boundary.xy[0]))
hole_poly_df.geometry.values[0]

In [ ]:
print(len(hole_poly_df.geometry.simplify(refine_tolerance).geometry.values[0].boundary.xy[0]))

hole_poly_df.geometry.simplify(refine_tolerance).values[0]

In [ ]:
hole_poly_df.geometry = hole_poly_df.geometry.simplify(refine_tolerance).copy()
hole_poly = []
for x,y in zip(hole_poly_df.geometry.values[0].boundary.xy[0],hole_poly_df.geometry.values[0].boundary.xy[1]):
    hole_poly.append((float(x), float(y)))

In [ ]:
holexy=hole_poly_df.geometry.centroid

In [ ]:
tri = Triangle(maximum_area=maximum_area, angle=33, model_ws=model_ws, 
               exe_name='triangle')
tri.add_polygon(domainpoly)
tri.add_polygon(hole_poly)
tri.add_hole([holexy.x.values,holexy.y.values])
tri.build(verbose=False)
fig = plt.figure(figsize=(15,15))
ax = plt.subplot(1, 1, 1, aspect='equal')
tri.plot(ax=ax);

## we can also refine around another polygon

In [ ]:
refined_poly_df = ch_cantons.loc[ch_cantons.NAME_1=='Uri'].copy()


In [ ]:
print(len(refined_poly_df.geometry.values[0].boundary.xy[0]))
refined_poly_df.geometry.values[0]

In [ ]:
refine_tolerance = 650
print(len(refined_poly_df.geometry.simplify(refine_tolerance).values[0].boundary.xy[0]))
refined_poly_df.geometry.simplify(refine_tolerance).values[0]

In [ ]:
refined_poly_df.geometry = refined_poly_df.geometry.simplify(refine_tolerance)

In [ ]:
ref_poly = []
for x,y in zip(refined_poly_df.geometry.values[0].boundary.xy[0],refined_poly_df.geometry.values[0].boundary.xy[1]):
    ref_poly.append((float(x), float(y)))

In [ ]:
ref_polyxy = refined_poly_df.geometry.centroid

In [ ]:
if os.path.exists(model_ws):
    shutil.rmtree(model_ws)
os.makedirs(model_ws)

tri = Triangle(angle=33, model_ws=model_ws, 
               exe_name='triangle')
tri.add_polygon(domainpoly)
tri.add_polygon(hole_poly)
tri.add_polygon(ref_poly)
tri.add_hole([holexy.x.values,holexy.y.values])

tri.add_region([600000, 150000], 1, maximum_area=maximum_area)
tri.add_region((ref_polyxy.x.values[0],ref_polyxy.y.values[0]), 0, maximum_area=maximum_area/10)
tri.build(verbose=False)
fig = plt.figure(figsize=(15,15))
ax = plt.subplot(1, 1, 1, aspect='equal')
tri.plot(ax=ax);

## we will need to add edgenodes for both the main domain and the hole to set boundaries

In [ ]:
# first the outer domain
edgenodes = []
for iedge in range(len(domainpoly)):
    nodes = tri.get_edge_cells(iedge)
    for n in nodes:
        if n not in edgenodes:
            edgenodes.append(n)  



In [ ]:
# now get the points around the hole
inset_nodes = []
for iedge in range(len(domainpoly),len(domainpoly)+len(hole_poly)):
    nodes = tri.get_edge_cells(iedge)
    for n in nodes:
        if n not in inset_nodes:
            inset_nodes.append(n)    

## now remake the model using the new triangle object for the grid

In [ ]:
name = 'mf'
sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                             exe_name='mf6',
                             sim_ws=model_ws)
tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                             perioddata=[[1.0, 1, 1.]])
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
ims = flopy.mf6.ModflowIms(sim, print_option='SUMMARY', complexity='complex', 
                           outer_hclose=1.e-5, inner_hclose=1.e-4)
cell2d = tri.get_cell2d()
vertices = tri.get_vertices()
xcyc = tri.get_xcyc()
nlay = 1
ncpl = tri.ncpl
nvert = tri.nvert
top = 1.
hk=0.8
botm = [0.]
dis = flopy.mf6.ModflowGwfdisv(gwf, nlay=nlay, ncpl=ncpl, nvert=nvert,
                               top=top, botm=botm, 
                               vertices=vertices, cell2d=cell2d)

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, k = hk, xt3doptions=[True], 
                              save_specific_discharge=True)
ic = flopy.mf6.ModflowGwfic(gwf)

chdlist = []
for icpl in edgenodes:
    h = 10.
    chdlist.append([(0, icpl), h])

for icpl in inset_nodes:
    h = 10.1
    chdlist.append([(0, icpl), h])
    
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chdlist)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=1e-8)
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord='{}.cbc'.format(name),
                            head_filerecord='{}.hds'.format(name),
                            saverecord=[('HEAD', 'LAST'),
                                        ('BUDGET', 'LAST')],
                            printrecord=[('HEAD', 'LAST'),
                                         ('BUDGET', 'LAST')])
sim.write_simulation()
success, buff = sim.run_simulation(report=True)

In [ ]:
fname = os.path.join(model_ws, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
fname = os.path.join(model_ws, name + '.cbc')
bdobj = flopy.utils.CellBudgetFile(fname, precision='double', verbose=False)
spdis = bdobj.get_data(text='DATA-SPDIS')[0]

fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
h=tri.plot(ax=ax, a=head[0, 0, :], cmap='jet', alpha=.3)
plt.colorbar(h, shrink=.5)

In [ ]:
fname = os.path.join(model_ws, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
fname = os.path.join(model_ws, name + '.cbc')
bdobj = flopy.utils.CellBudgetFile(fname, precision='double', verbose=False)
spdis = bdobj.get_data(text='DATA-SPDIS')[0]

fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
h=tri.plot(ax=ax, a=head[0, 0, :], cmap='jet', alpha=.3)
plt.colorbar(h, shrink=.5)

In [ ]:
# with vectors
fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
pmv = flopy.plot.PlotMapView(model=gwf, ax=ax)
h=pmv.plot_array(head[0, :, :], cmap='jet', edgecolor='0.1', alpha=0.3)
pmv.plot_specific_discharge(spdis)
plt.colorbar(h,shrink=0.4)